# Test Theis Storage

In [ ]:
import numpy as np
from scipy.integrate import quad, solve_ivp
from scipy.special import exp1

In [ ]:
Q = -100
npor = 0.3
k = 10
H = 10
T = k * H
Ss = 1e-4
S = Ss * H

In [ ]:
def headtheis(r, t, T=T, S=S, Q=Q):
    u = r**2 * S / (4 * T * t)
    h = -Q / (4 * np.pi * T) * exp1(u)
    return h


def voltheis(r, t, T, S, Q):
    u = r**2 * S / (4 * T * t)
    h = -Q / (4 * np.pi * T) * exp1(u)
    vol = h * 2 * np.pi * r
    return vol * S

In [ ]:
# demonstrate headtheis works correctly
quad(voltheis, a=1e-3, b=10000, args=(10, T, S, Q))  # gives 1000 m^3

In [ ]:
def volume(r, t, T=T, S=S, Q=Q):
    return quad(voltheis, a=1e-3, b=r, args=(10, T, S, Q))[0] + npor * np.pi * (r**2)

In [ ]:
def vxytheis(t, xy):
    x, y = xy
    r = np.sqrt(x**2 + y**2)
    u = S * r**2 / (4 * T * t)
    Qr = -Q / (2 * np.pi) / r * np.exp(-u)
    vr = Qr / (H * npor)
    vx = vr * x / r
    vy = vr * y / r
    return np.array([vx, vy])

In [ ]:
def vxytheisnew(t, xy):
    x, y = xy
    r = np.sqrt(x**2 + y**2)
    u = S * r**2 / (4 * T * t)
    Qr = -Q / (2 * np.pi) / r * np.exp(-u)
    vr = Qr / (H * (npor + S * headtheis(x, t)))
    vx = vr * x / r
    vy = vr * y / r
    return np.array([vx, vy])

In [ ]:
# trace pathline for 10 days
path0 = solve_ivp(vxytheis, (1e-5, 10), y0=[1e-5, 0], method="DOP853")
R0 = path0.y[0, -1]
R0, np.pi * R0**2 * npor * H

In [ ]:
path0 = solve_ivp(vxytheisnew, (1e-5, 10), y0=[1e-5, 0], method="DOP853")
R1 = path0.y[0, -1]
R1, np.pi * R1**2 * npor * H

In [ ]:
headtheis(5, 10) * S

In [ ]:
npor * H

In [ ]:
R = path0.y[0, -1]
print("R, volume", R, np.pi * R**2 * npor * H)

In [ ]:
from scipy.integrate import quad

quad(headtheis, 1e-5, R, args=(100, T, S, Q))[0] + np.pi * R**2 * npor * H

In [ ]:
t0 = 0
t1 = 10


def vxytheis2(t, xy):
    x, y = xy
    r = np.sqrt(x**2 + y**2)
    u1 = S * r**2 / (4 * T * (t - t0))
    u2 = S * r**2 / (4 * T * (t - t1))
    Qr = -Q / (2 * np.pi) / r * np.exp(-u1)
    if t >= t1:
        Qr += 2 * Q / (2 * np.pi) / r * np.exp(-u2)
    vr = Qr / (H * npor)
    vx = vr * x / r
    vy = vr * y / r
    return np.array([vx, vy])

In [ ]:
path1 = solve_ivp(vxytheis2, (10 + 1e-6, 20), y0=[R, 0], method="DOP853")
path1.y[0, -1]

In [ ]:
path1

In [ ]:
vxytheis2(12 + 1e-6, (10, 0))